<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Losses" data-toc-modified-id="Losses-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Losses</a></span></li><li><span><a href="#VITS-Trainer" data-toc-modified-id="VITS-Trainer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>VITS Trainer</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# default_exp trainer.vits

In [ ]:
# export
import json
import os
from pathlib import Path
from pprint import pprint

import torch
from torch.cuda.amp import autocast, GradScaler
import torch.distributed as dist
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

from uberduck_ml_dev.models.common import MelSTFT
from uberduck_ml_dev.utils.plot import (
    plot_attention,
    plot_gate_outputs,
    plot_spectrogram,
)
from uberduck_ml_dev.text.util import text_to_sequence, random_utterance
from uberduck_ml_dev.text.symbols import symbols_with_ipa
from uberduck_ml_dev.trainer.base import TTSTrainer

from uberduck_ml_dev.models.vits import (
    DEFAULTS,
    MultiPeriodDiscriminator,
    SynthesizerTrn,
)
from uberduck_ml_dev.data_loader import (
    TextAudioSpeakerLoader,
    TextAudioSpeakerCollate,
    DistributedBucketSampler,
)
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.utils.utils import slice_segments, clip_grad_value_

# Losses

In [ ]:
# export


def feature_loss(fmap_r, fmap_g):
    loss = 0
    for dr, dg in zip(fmap_r, fmap_g):
        for rl, gl in zip(dr, dg):
            rl = rl.float().detach()
            gl = gl.float()
            loss += torch.mean(torch.abs(rl - gl))

    return loss * 2


def discriminator_loss(disc_real_outputs, disc_generated_outputs):
    loss = 0
    r_losses = []
    g_losses = []
    for dr, dg in zip(disc_real_outputs, disc_generated_outputs):
        dr = dr.float()
        dg = dg.float()
        r_loss = torch.mean((1 - dr) ** 2)
        g_loss = torch.mean(dg ** 2)
        loss += r_loss + g_loss
        r_losses.append(r_loss.item())
        g_losses.append(g_loss.item())

    return loss, r_losses, g_losses


def generator_loss(disc_outputs):
    loss = 0
    gen_losses = []
    for dg in disc_outputs:
        dg = dg.float()
        l = torch.mean((1 - dg) ** 2)
        gen_losses.append(l)
        loss += l

    return loss, gen_losses


def kl_loss(z_p, logs_q, m_p, logs_p, z_mask):
    """
    z_p, logs_q: [b, h, t_t]
    m_p, logs_p: [b, h, t_t]
    """
    z_p = z_p.float()
    logs_q = logs_q.float()
    m_p = m_p.float()
    logs_p = logs_p.float()
    z_mask = z_mask.float()

    kl = logs_p - logs_q - 0.5
    kl += 0.5 * ((z_p - m_p) ** 2) * torch.exp(-2.0 * logs_p)
    kl = torch.sum(kl * z_mask)
    l = kl / torch.sum(z_mask)
    return l

# VITS Trainer

In [ ]:
# export


class VITSTrainer(TTSTrainer):
    REQUIRED_HPARAMS = [
        "betas",
        "eps",
        "training_audiopaths_and_text",
        "val_audiopaths_and_text",
    ]

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.mel_stft = MelSTFT()

    def log_training(self):
        print("log training placeholder...")

    def log_validation(self):
        pass

    def warm_start(self):
        pass

    def _evaluate(self, generator, val_loader):
        pass

    def _train_and_evaluate(
        self, epoch, nets, optims, schedulers, scaler: GradScaler, loaders
    ):
        net_g, net_d = nets
        optim_g, optim_d = optims
        scheduler_g, scheduler_d = schedulers
        train_loader, val_loader = loaders
        train_loader.batch_sampler.set_epoch(epoch)
        net_g.train()
        net_d.train()
        for batch_idx, (
            x,
            x_lengths,
            spec,
            spec_lengths,
            y,
            y_lengths,
            speakers,
        ) in enumerate(train_loader):
            if self.device == "cuda":
                x, x_lengths = x.cuda(self.rank, non_blocking=True), x_lengths.cuda(
                    self.rank, non_blocking=True
                )
                y, y_lengths = y.cuda(self.rank, non_blocking=True), y_lengths.cuda(
                    self.rank, non_blocking=True
                )
                spec, spec_lengths = spec.cuda(
                    self.rank, non_blocking=True
                ), spec_lengths.cuda(self.rank, non_blocking=True)
                speakers = speakers.cuda(self.rank, non_blocking=True)
            with autocast(enabled=self.fp16_run):
                (
                    y_hat,
                    l_length,
                    attn,
                    ids_slice,
                    x_mask,
                    z_mask,
                    (z, z_p, m_p, logs_p, m_q, logs_q),
                ) = net_g(x, x_lengths, spec, spec_lengths, speakers)
                mel = self.mel_stft.spec_to_mel(spec)
                y_mel = slice_segments(
                    mel, ids_slice, self.segment_size // self.hop_length
                )
                y_hat_mel = self.mel_stft.mel_spectrogram(y_hat.squeeze(1))
                y = slice_segments(y, ids_slice * self.hop_length, self.segment_size)

                # Discriminator
                y_d_hat_r, y_d_hat_g, _, _ = net_d(y, y_hat.detach())
                with autocase(enabled=False):
                    loss_disc, losses_disc_r, losses_disc, g = discriminator_loss(
                        y_d_hat_r, y_d_hat_g
                    )
                    loss_disc_all = loss_disc
            optim_d.zero_grad()
            scaler.scale(loss_disc_all).backward()
            scaler.unscale_(optim_d)
            grad_norm_d = clip_grad_value_(net_d.parameters(), None)
            scaler.step(optim_d)

            with autocast(enabled=self.fp16_run):
                # Generator
                y_d_hat_r, y_d_hat_g, fmap_r, fmap_g = net_d(y, y_hat)
                with autocast(enabled=False):
                    loss_dur = torch.sum(l_length.float())
                    loss_mel = F.l1_loss(y_mel, y_hat_mel) * self.c_mel
                    loss_kl = kl_loss(z_p, logs_q, m_p, logs_p, z_mask) * self.c_kl

                    loss_fm = feature_loss(fmap_r, fmap_g)
                    loss_gen, losses_gen = generator_loss(y_d_hat_g)
                    loss_gen_all = loss_gen + loss_fm + loss_mel + loss_dur + loss_kl
            optim_g.zero_grad()
            scaler.scale(loss_gen_all).backward()
            scaler.unscale_(optim_g)
            grad_norm_g = clip_grad_value_(enet_g.parameters(), None)
            scaler.step(optim_g)
            scaler.update()

            self.log_training()
        self._evaluate(net_g, val_loader)

    def train(self):
        train_dataset = TextAudioSpeakerLoader(
            self.training_audiopaths_and_text, self.hparams
        )
        train_sampler = DistributedBucketSampler(
            train_dataset,
            self.batch_size,
            [32, 300, 400, 500, 600, 700, 800, 900, 1000],
            num_replicas=self.world_size,
            rank=self.rank,
            shuffle=True,
        )
        collate_fn = TextAudioSpeakerCollate()
        train_loader = DataLoader(
            train_dataset,
            num_workers=8,
            shuffle=False,
            pin_memory=True,
            collate_fn=collate_fn,
            batch_sampler=train_sampler,
        )
        if rank == 0:
            val_dataset = TextAudioSpeakerLoader(
                self.val_audiopaths_and_text, self.hparams
            )
            val_loader = DataLoader(
                val_dataset,
                num_workers=8,
                shuffle=False,
                batch_size=self.batch_size,
                pin_memory=True,
                drop_last=False,
                collate_fn=collate_fn,
            )

        model_kwargs = {k: v for k, v in DEFAULTS.values().items() if hasattr(self, k)}
        net_g = SynthesizerTrn(
            len(symbols_with_ipa),
            self.filter_length // 2 + 1,
            self.segment_size // self.hop_length,
            n_speakers=self.n_speakers,
            **model_kwargs,
        )
        net_id = MultiPeriodDiscriminator(self.use_spectral_norm)
        if self.device == "cuda":
            net_g = net_g.cuda(self.rank)
            net_d = net_d.cuda(self.rank)

        optim_g = torch.optim.AdamW(
            net_g.parameters(),
            self.learning_rate,
            betas=self.betas,
            eps=self.eps,
        )
        optim_d = torch.optim.AdamW(
            net_d.parameters(), self.learning_rate, betas=self.betas, eps=self.eps
        )
        if self.distributed_run:
            net_g = DDP(net_g, device_ids=[self.rank])
            net_d = DDP(net_d, device_ids=[self.rank])

        start_epoch = 0
        if self.warm_start_name:
            # TODO
            pass

        scheduler_g = ExponentialLR(
            optim_g, gamma=self.lr_decay, last_epoch=start_epoch - 1
        )
        scheduler_d = ExponentialLR(
            optim_d, gamma=self.lr_decay, last_epoch=start_epoch - 1
        )
        scaler = GradScaler(enabled=self.fp16_run)

        for epoch in range(start_epoch, self.epochs):
            self._train_and_evaluate(
                epoch,
                [net_g, net_d],
                [optim_g, optim_d],
                [scheduler_g, scheduler_d],
                scaler,
                [train_loader, val_loader],
            )
            if epoch % self.epochs_per_checkpoint == 0:
                self.save_checkpoint(f"vits_{epoch}")